# Tagging and Extraction Using OpenAI functions

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from typing import List
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [29]:
class Tagging(BaseModel):
    """Tag the piece of text with particular info."""
    sentiment: str = Field(description="sentiment of text, should be `pos`, `neg`, or `neutral`")
    language: str = Field(description="language of text (should be ISO 639-1 code)")
    tense: str = Field(description="verb tense of the text (should be `past`, `present`, or `future`.)")
    english_translation: str = Field(description="english language translation of the text.")

In [30]:
convert_pydantic_to_openai_function(Tagging)

{'name': 'Tagging',
 'description': 'Tag the piece of text with particular info.',
 'parameters': {'title': 'Tagging',
  'description': 'Tag the piece of text with particular info.',
  'type': 'object',
  'properties': {'sentiment': {'title': 'Sentiment',
    'description': 'sentiment of text, should be `pos`, `neg`, or `neutral`',
    'type': 'string'},
   'language': {'title': 'Language',
    'description': 'language of text (should be ISO 639-1 code)',
    'type': 'string'},
   'tense': {'title': 'Tense',
    'description': 'verb tense of the text (should be `past`, `present`, or `future`.)',
    'type': 'string'},
   'english_translation': {'title': 'English Translation',
    'description': 'english language translation of the text.',
    'type': 'string'}},
  'required': ['sentiment', 'language', 'tense', 'english_translation']}}

In [31]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

In [32]:
model = ChatOpenAI(temperature=0)

In [33]:
tagging_functions = [convert_pydantic_to_openai_function(Tagging)]

In [34]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Think carefully, and then tag the text as instructed"),
    ("user", "{input}")
])

In [35]:
model_with_functions = model.bind(
    functions=tagging_functions,
    function_call={"name": "Tagging"}
)

In [36]:
tagging_chain = prompt | model_with_functions

In [37]:
tagging_chain.invoke({"input": "I love langchain"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'Tagging', 'arguments': '{"sentiment":"pos","language":"en","tense":"present","english_translation":"I love langchain"}'}})

In [38]:
tagging_chain.invoke({"input": "non mi piace questo cibo"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'Tagging', 'arguments': '{"sentiment":"neg","language":"it","tense":"present","english_translation":"I don\'t like this food"}'}})

In [39]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

In [40]:
tagging_chain = prompt | model_with_functions | JsonOutputFunctionsParser()

In [41]:
tagging_chain.invoke({"input": "non mi piace questo cibo"})

{'sentiment': 'neg',
 'language': 'it',
 'tense': 'present',
 'english_translation': "I don't like this food"}

In [43]:
tagging_chain.invoke({"input": "no me mires mas."})

{'sentiment': 'neg',
 'language': 'es',
 'tense': 'present',
 'english_translation': "Don't look at me anymore."}

In [44]:
tagging_chain.invoke({"input": "das war nicht so toll."})

{'sentiment': 'neg',
 'language': 'de',
 'tense': 'past',
 'english_translation': 'That was not so great.'}

In [46]:
tagging_chain.invoke({"input": "je serais la."})

{'sentiment': 'neutral',
 'language': 'fr',
 'tense': 'future',
 'english_translation': 'I will be there.'}

## Extraction

Extraction is similar to tagging, but used for extracting multiple pieces of information.

In [110]:
from typing import Optional
class Person(BaseModel):
    """Information about a person."""
    first_name: str = Field(description="person's first name")
    age: Optional[int] = Field(description="person's age")
    last_name: str = Field(description="person's last name.")

In [111]:
class Information(BaseModel):
    """Information to extract."""
    people: List[Person] = Field(description="List of info about people")

In [112]:
convert_pydantic_to_openai_function(Information)

{'name': 'Information',
 'description': 'Information to extract.',
 'parameters': {'title': 'Information',
  'description': 'Information to extract.',
  'type': 'object',
  'properties': {'people': {'title': 'People',
    'description': 'List of info about people',
    'type': 'array',
    'items': {'title': 'Person',
     'description': 'Information about a person.',
     'type': 'object',
     'properties': {'first_name': {'title': 'First Name',
       'description': "person's first name",
       'type': 'string'},
      'age': {'title': 'Age',
       'description': "person's age",
       'type': 'integer'},
      'last_name': {'title': 'Last Name',
       'description': "person's last name.",
       'type': 'string'}},
     'required': ['first_name', 'last_name']}}},
  'required': ['people']}}

In [113]:
extraction_functions = [convert_pydantic_to_openai_function(Information)]
extraction_model = model.bind(functions=extraction_functions, function_call={"name": "Information"})

In [114]:
extraction_model.invoke("Joe is 30, his mom is Martha")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'Information', 'arguments': '{"people":[{"first_name":"Joe","age":30},{"first_name":"Martha"}]}'}})

In [115]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("human", "{input}")
])

In [116]:
extraction_chain = prompt | extraction_model

In [117]:
extraction_chain.invoke({"input": "Joe Biden is 30, his mom is Martha"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'Information', 'arguments': '{"people":[{"first_name":"Joe","age":30,"last_name":"Biden"},{"first_name":"Martha","last_name":"Biden"}]}'}})

In [118]:
extraction_chain = prompt | extraction_model | JsonOutputFunctionsParser()

In [119]:
extraction_chain.invoke({"input": "Joe Brown is 30, his mom is Martha Brown"})

{'people': [{'first_name': 'Joe', 'age': 30, 'last_name': 'Brown'},
  {'first_name': 'Martha', 'last_name': 'Brown'}]}

In [120]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

In [121]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="people")

In [123]:
extraction_chain.invoke({"input": "Joe White is 30, his mom is Martha."})

[{'first_name': 'Joe', 'age': 30, 'last_name': 'White'},
 {'first_name': 'Martha', 'last_name': 'Unknown'}]

## Doing it for real

We can apply tagging to a larger body of text.

For example, let's load this blog post and extract tag information from a sub-set of the text.

In [105]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
documents = loader.load()

In [106]:
doc = documents[0]

In [107]:
page_content = doc.page_content[:10000]

In [108]:
print(page_content[:1000])







LLM Powered Autonomous Agents | Lil'Log







































Lil'Log

















|






Posts




Archive




Search




Tags




FAQ




emojisearch.app









      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


 


Table of Contents



Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three: Tool Use

Case Studies

Scientific Discovery Agent

Generative Agents Simulation

Proof-of-Concept Examples


Challenges

Citation

References





Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful gene

In [109]:
class Overview(BaseModel):
    """Overview of a section of text."""
    summary: str = Field(description="Provide a concise summary of the content.")
    language: str = Field(description="Provide the language that the content is written in.")
    keywords: str = Field(description="Provide keywords related to the content.")    

In [124]:
overview_tagging_function = [
    convert_pydantic_to_openai_function(Overview)
]
tagging_model = model.bind(
    functions=overview_tagging_function,
    function_call={"name":"Overview"}
)
tagging_chain = prompt | tagging_model | JsonOutputFunctionsParser()

In [125]:
tagging_chain.invoke({"input": page_content})

{'summary': 'This text discusses the concept of building autonomous agents powered by LLM (large language model) as the core controller. It covers components such as planning, memory, and tool use, along with techniques like task decomposition and self-reflection. Various proof-of-concept examples and challenges are also mentioned.',
 'language': 'English',
 'keywords': 'LLM, autonomous agents, planning, memory, tool use, task decomposition, self-reflection, proof-of-concept, challenges'}

In [126]:
class Paper(BaseModel):
    """Information about papers mentioned."""
    title: str
    author: Optional[str]


class Info(BaseModel):
    """Information to extract"""
    papers: List[Paper]

In [127]:
paper_extraction_function = [
    convert_pydantic_to_openai_function(Info)
]
extraction_model = model.bind(
    functions=paper_extraction_function, 
    function_call={"name":"Info"}
)
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")

In [128]:
extraction_chain.invoke({"input": page_content})

[{'title': 'LLM Powered Autonomous Agents', 'author': 'Lilian Weng'}]

In [129]:
template = """A article will be passed to you. Extract from it all papers that are mentioned by this article follow by its author. 

Do not extract the name of the article itself. If no papers are mentioned that's fine - you don't need to extract any! Just return an empty list.

Do not make up or guess ANY extra information. Only extract what exactly is in the text."""

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", "{input}")
])

In [154]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")

In [155]:
extraction_chain.invoke({"input": page_content})

[{'title': 'Chain of thought (CoT; Wei et al. 2022)'},
 {'title': 'Tree of Thoughts (Yao et al. 2023)'},
 {'title': 'LLM+P (Liu et al. 2023)'},
 {'title': 'ReAct (Yao et al. 2023)'},
 {'title': 'Reflexion (Shinn & Labash 2023)'},
 {'title': 'Chain of Hindsight (CoH; Liu et al. 2023)'},
 {'title': 'Algorithm Distillation (AD; Laskin et al. 2023)'}]

In [132]:
extraction_chain.invoke({"input": "hi"})

[]

In [133]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_overlap=0)

In [134]:
splits = text_splitter.split_text(doc.page_content)

In [135]:
len(splits)

15

In [136]:
def flatten(matrix):
    flat_list = []
    for row in matrix:
        flat_list += row
    return flat_list

In [137]:
flatten([[1, 2], [3, 4]])

[1, 2, 3, 4]

In [138]:
print(splits[0])

LLM Powered Autonomous Agents | Lil'Log







































Lil'Log

















|






Posts




Archive




Search




Tags




FAQ




emojisearch.app









      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


 


Table of Contents



Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three: Tool Use

Case Studies

Scientific Discovery Agent

Generative Agents Simulation

Proof-of-Concept Examples


Challenges

Citation

References





Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general pr

In [139]:
from langchain.schema.runnable import RunnableLambda

In [175]:
prep = RunnableLambda(
    lambda x: [{"input": doc} for doc in text_splitter.split_text(x)]
)

In [176]:
chain = prep | extraction_chain.map() | flatten

In [177]:
chain.invoke(doc.page_content)

KeyError: 'papers'